# Movement similarity

https://github.com/move-ucsb/move-similarity

Author: Zijian Wan

In [ ]:
import os
import copy
import math
import pickle
import random
from datetime import datetime

from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from scipy import stats
from sklearn import metrics, preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering, DBSCAN, SpectralClustering

## Hierarchical clustering

The pair-wise distances based on various measures, including Frechet, DTW, Hausdorff, LCSS, and NWED, are precomputed using the **trajectory_distance** module, which is available on [GitHub](https://github.com/bguillouet/traj-dist). 

In [ ]:
def traj_clustering(metric, n_clu_lowbd, n_clu_highbd, trajs=trajs):
    '''
    Cluster trajectories using different metrics.

    Parameters
    ----------
    metric : str
    trajs : list
        a list of trajectories, a trajectory is a n*2 np.array (x, y)
    n_clu_lowbd : int
        lower bound of the number of clusters
    n_clu_highbd : TYPE
        upper bound of the number of clusters

    Returns
    -------
    labels_df : pd.dataframe

    '''
    n_cols = n_clu_highbd - n_clu_lowbd + 1
    
    # load the distant array file
    file_name = "mid_migr_lat_30N_tv_" + metric + "_arr"
    # open the file for reading
    file_object = open(file_name, 'rb')
    # load the object from the file into var c
    dist_arr = pickle.load(file_object)
    
    # clustering
    labels = np.zeros((len(trajs), n_cols)).astype(int)
    count = 0
    n_clu_list = []
    for n_clusters in range(n_clu_lowbd, n_clu_highbd+1):
        cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='average')
        clu_results = cluster.fit(dist_arr)
        labels[:, count] = clu_results.labels_
        count += 1
        n_clu_list.append(metric + '_' + str(n_clusters))
    
    # output
    labels_df = pd.DataFrame(data=labels, columns=n_clu_list)
    # add 'TrajID' column
    trajID = list(range(len(trajs)))
    labels_df['TrajID'] = trajID
    # rearrange column order
    cols = labels_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    labels_df = labels_df[cols]
    
    return labels_df

In [ ]:
labels_df = traj_clustering(metric='frechet', n_clu_lowbd=2, n_clu_highbd=9)
cluster_df = copy.deepcopy(labels_df)
# dtw
labels_df = traj_clustering(metric='dtw', n_clu_lowbd=2, n_clu_highbd=9)
cluster_df = cluster_df.merge(labels_df, on='TrajID')
# hausdorff
labels_df = traj_clustering(metric='hausdorff', n_clu_lowbd=2, n_clu_highbd=9)
cluster_df = cluster_df.merge(labels_df, on='TrajID')
# lcss
labels_df = traj_clustering(metric='lcss', n_clu_lowbd=2, n_clu_highbd=9)
cluster_df = cluster_df.merge(labels_df, on='TrajID')
# nwed
labels_df = traj_clustering(metric='nwed', n_clu_lowbd=2, n_clu_highbd=9)
cluster_df = cluster_df.merge(labels_df, on='TrajID')

Put clustering results based on different distance metrics together for further analysis

* Frechet
* DTW
* Hausdorff
* LCSS
* NWED

Also include turkey vulture id, name, and fall/spring migration

In [ ]:
tags = []
names = []
migr_status = []
fall_begin = datetime(month=10, day=1, year=2000).strftime('%m/%d')
fall_end = datetime(month=12, day=5, year=2000).strftime('%m/%d')
spring_begin = datetime(month=3, day=15, year=2000).strftime('%m/%d')
spring_end = datetime(month=5, day=5, year=2000).strftime('%m/%d')

for tji in range(cluster_df.shape[0]):
    pti = trajPtID[tji][1][0]
    
    tags.append(pts_df.loc[pti, 'tag_local_identifier'])
    names.append(pts_df.loc[pti, 'individual_local_identifier'])
    
    pti_dt = pts_df.loc[pti, 'study_local_timestamp']
    pti_date = datetime.strptime(pti_dt, '%m/%d/%Y %H:%M:%S').strftime('%m/%d')
    if fall_begin <= pti_date <= fall_end:
        migr_status.append('Fall')
    elif spring_begin <= pti_date <= spring_end:
        migr_status.append('Spring')

# add columns to cluster_df
cluster_df['tag'] = tags
cluster_df['name'] = names
cluster_df['migr_status'] = migr_status
# rearrange column order
cols = cluster_df.columns.tolist()
cols = cols[-3:] + cols[:-3]
cluster_df = cluster_df[cols]

In [ ]:
def Reorder_clu_id(labels, n_clusters):
    '''
    Reorder the cluster IDs so that the order of the cluster IDs is in the order of
    the number of trajectories it contains.
        e.g. 
        cluster ID 0: most
        cluster ID 1: second most
        ...
        cluster ID n: least

    Parameters
    ----------
    labels : np.array
        labels of clusters
    n_clusters : int
        number of clusters

    Returns
    -------
    re_labels : np.array
        labels reordered

    '''
    n_rows = len(labels)
    
    # count the cluster ids
    contrast = np.zeros((n_clusters, 3))
    contrast_df = pd.DataFrame(contrast, columns=['clu_id', 'count', 'reordered'])
    contrast_df.loc[:, 'clu_id'] = list(range(n_clusters))
    for rowi in range(n_rows):
        clu_id = labels[rowi]
        contrast_df.loc[clu_id, 'count'] = contrast_df.loc[clu_id, 'count'] + 1
    
    # sort by count
    contrast_df = contrast_df.sort_values(by=['count'], ascending=False)
    contrast_df.loc[:, 'reordered'] = list(range(n_clusters))
    
    # output
    re_labels = np.zeros(n_rows)
    for rowi in range(n_rows):
        clu_id_original = labels[rowi]
        clu_id_reordered = contrast_df.loc[clu_id_original, 'reordered']
        re_labels[rowi] = clu_id_reordered
    
    re_labels = re_labels.astype('int')
    return re_labels
    

def Reorder_cluster_df(cluster_df, cols):
    '''
    Reorder a cluster_df

    Parameters
    ----------
    cluster_df : pd.DataFrame
        cluster IDs (and other descriptive information)
    cols : list
        column numbers (of cluster IDs) that need to be reordered
        e.g., [3, 10]

    Returns
    -------
    re_cluster_df : pd.DataFrame
        cluster_df reordered 

    '''
    col_names = cluster_df.columns.tolist()
    
    # output
    re_cluster_df = copy.deepcopy(cluster_df)
    
    # reorder each column
    for coli in cols:
        # labels (np.array)
        labels = cluster_df.iloc[:, coli].to_numpy()
        
        # number of clusters
        col_name = col_names[coli]
        n_clu_index = col_name.find('_') + 1
        n_clusters = int(col_name[n_clu_index:])
        
        # reorder the column
        re_labels = Reorder_clu_id(labels, n_clusters)
        re_cluster_df.iloc[:, coli] = re_labels
    
    return re_cluster_df

In [ ]:
cols = list(range(4, cluster_df.shape[1]))
cluster_df = Reorder_cluster_df(cluster_df, cols)

In [ ]:
def Metric_compare(metric, n_clusters, cluster_df=cluster_df):
    '''
    Examine the trajectories in different clusters.
    '''
    # find the corresponding trajectories
    labels = cluster_df.loc[:, metric].to_numpy()
    label_trajs = []
    for clui in range(n_clusters):
        traj_index = np.where(labels == clui)[0]
        label_trajs.append(traj_index)
    
    avg_n_pts = []  # average number of pts in a trajectory
    avg_len_km = []  # average trajectory lengths in kilometers
    fall_count = []  # fall migration count
    spring_count = []  # spring migration count
    avg_start_time = []  # average start time (local)
    avg_end_time = []  # average end time (local)
    
    df_indexes = []  # indexes in the output pd.DataFrame
    
    for clui in range(n_clusters):
        n_trajs = len(label_trajs[clui])  # number of trajectories in the cluster
        
        n_pts = []
        traj_len = []
        
        # if it is a fall migration trajectory, +1
        # if spring, +0
        fall_sum = 0
        
        start_timestamps = []
        end_timestamps = []
        
        for tji in label_trajs[clui]:
            # number of points
            n_pts.append(trajPtID[tji][2]) 
            traj_len.append(traj_len_km[tji])
            # migration status
            if cluster_df.loc[tji, 'migr_status'] == 'Fall':
                fall_sum += 1
            # start and end time
            start_pt = trajPtID[tji][1][0]
            end_pt = trajPtID[tji][1][-1]
            
            real_start_time = datetime.strptime(pts_df.loc[start_pt, 'study_local_timestamp'],
                                                '%m/%d/%Y %H:%M:%S')
            # year should not be considered
            start_time = real_start_time.replace(year=2000)
            start_time = datetime.timestamp(start_time)
            
            real_end_time = datetime.strptime(pts_df.loc[end_pt, 'study_local_timestamp'],
                                                '%m/%d/%Y %H:%M:%S')
            # year should not be considered
            end_time = real_end_time.replace(year=2000)
            end_time = datetime.timestamp(end_time)
            
            start_timestamps.append(start_time)
            end_timestamps.append(end_time)
        
        avg_n_pts.append(round(np.mean(n_pts)))  # average number of points
        avg_len_km.append(round(np.mean(traj_len), 3))  # average trajectory lengths in kilometers
        fall_count.append(fall_sum)  # fall migration count
        spring_count.append(n_trajs - fall_sum)  # spring migration count
        avg_start_timestamp = int(np.mean(start_timestamps))
        avg_end_timestamp = int(np.mean(end_timestamps))
        # average start time
        avg_start_time.append(datetime.fromtimestamp(avg_start_timestamp).strftime("%m/%d %H:%M:%S"))
        # average end time
        avg_end_time.append(datetime.fromtimestamp(avg_end_timestamp).strftime("%m/%d %H:%M:%S"))
        
        df_indexes.append(metric + '_' + str(clui))   
    
        new_df = pd.DataFrame(list(zip(avg_n_pts, avg_len_km, fall_count, spring_count,
                                         avg_start_time, avg_end_time)),
                                         columns = ['avg_n_pts', 'avg_len_km', 'fall_count',
                                                    'spring_count', 'avg_start_time',
                                                    'avg_end_time'],
                                         index=df_indexes)

    return new_df

### Initial clustering evaluation using Silhoette Coefficient

In [ ]:
def metric_silh(mtName, n_clu_lowbd, n_clu_highbd, cluster_df=cluster_df):
    '''
    Compute the Silhouette Coefficient of the initial clustering result.

    Parameters
    ----------
    mtName : str
        Name of the metric, e.g., 'frechet'.
    n_clu_lowbd : int
        The lower bound of the number of clusters, e.g., 2.
    n_clu_highbd : int
        The upper bound of the number of clusters, e.g., 9.
    cluster_df : pd.DataFrame, optional
        The dataframe storing the clustering results. The default is cluster_df.

    Returns
    -------
    silh : float
        Computed Silhouette Coefficient.
    '''
    # read trajectory data
    file_name = "mid_migr_lat_30N_tv_" + mtName + "_arr"
    file_object = open(file_name, 'rb')
    dist_arr = pickle.load(file_object)
    
    # normalization
    min_max_scaler = preprocessing.MinMaxScaler()
    dist_one_column = dist_arr.reshape([-1,1])
    norm_dist_one_column = min_max_scaler.fit_transform(dist_one_column)
    norm_dist_arr = norm_dist_one_column.reshape(dist_arr.shape)
    
    # read labels 
    n_clu = []
    for n_clui in range(n_clu_lowbd, n_clu_highbd+1):
        n_clu.append(mtName + '_' + str(n_clui))
    labels = cluster_df.loc[:, n_clu]
    
    # compute sc
    silh = []
    for n_clui in range(labels.shape[1]):
        silh.append(metrics.silhouette_score(norm_dist_arr, labels.iloc[:, n_clui], 
                                                 metric='precomputed'))
    
    return silh

In [ ]:
n_clu_lowbd = 2
n_clu_highbd = 9
fre_silh = metric_silh('frechet', n_clu_lowbd, n_clu_highbd, cluster_df)
dtw_silh = metric_silh('dtw', n_clu_lowbd, n_clu_highbd, cluster_df)
hausdorff_silh = metric_silh('hausdorff', n_clu_lowbd, n_clu_highbd, cluster_df)
lcss_silh = metric_silh('lcss', n_clu_lowbd, n_clu_highbd, cluster_df)
nwed_silh = metric_silh('nwed', n_clu_lowbd, n_clu_highbd, cluster_df)

### Hierachical clustering

In [ ]:
def initialClui(metric, n_clusters, clui, current_cluster_df=cluster_df):
    # initial cluster clui
    clusters = []
    for ci in range(n_clusters):
        traj_index = current_cluster_df[metric + '_' + str(n_clusters)] == ci
        clu_trajs = current_cluster_df.loc[traj_index, 'TrajID'].to_list()
        n_trajs = len(clu_trajs)
        clusters.append([metric + '_' + str(clui), 'n_trajs=' + str(n_trajs), clu_trajs])
    
    clui_trajs = clusters[clui][2]
    
    df_idx = []
    current_trajs = current_cluster_df.loc[:, 'TrajID'].to_list()
    for ci in clui_trajs:
        df_idx.append(current_trajs.index(ci))
    
    clui_df = current_cluster_df.loc[df_idx, ['tag', 'name', 'migr_status', 'TrajID', 
                                         metric + '_' + str(n_clusters)]].reset_index(
                                             drop=True)
    clui_df.rename(columns={metric + '_' + str(n_clusters): metric + '-' + str(n_clusters)}, inplace=True)

    # add manual labels for later comparison
    # label 0-n_tv based on the frequency
    tags = clui_df['tag'].to_list()
    tags_sorted = sorted(tags, key = tags.count, reverse = True)
    tags_unique = []
    for tagi in tags_sorted:
        if tagi not in tags_unique:
            tags_unique.append(tagi)
    for count, tagi in enumerate(tags_unique):
        clui_df.loc[clui_df['tag'] == tagi, 'tag_label'] =  count
    
    return clusters, clui_trajs, clui_df


def zoomInNCluster(metric, n_clusters, clui, current_cluster_df=cluster_df):
    '''
    Zoom in cluster
    perform clustering again based on the results of initial clustering
    
    Parameters
    ---------
    metric : str
        metric of initial clustering, e.g., 'frechet'
    n_clusters : int
        number of initial clusters 
    clui : int
        cluster id, e.g., 0
    current_cluster_df : pd.DataFrame
        initial clustering results

    Returns
    -------
    cluster_2nd_df : pd.DataFrame
        clustering results based on 5 different metrics

    '''
    clusters, clui_trajs, clui_df = initialClui(metric, n_clusters, clui, current_cluster_df)
    
    # secondary clustering
    # frechet, dtw, hausdorff, lcss, nwed
    # n_clusters: 2-9
    n_start = 2
    
    # check n_trajs
    n_trajs = len(clui_trajs)
    if n_trajs > 9:
        n_end = 9
    else:
        n_end = n_trajs - 1
    n_clu_col = 5 * (n_end - n_start + 1)
    
    labels = np.zeros((len(clui_trajs), n_clu_col)).astype(int)
    dist_metrics = ['frechet', 'dtw', 'hausdorff', 'lcss', 'nwed']
    n_clu_list = []  # column names
    count = 0
    
    for meti in dist_metrics:
        # read precomputed distance array
        file_name = "mid_migr_lat_30N_tv_" + meti + "_arr"
        file_object = open(file_name, 'rb')
        dist_arr = pickle.load(file_object)
        
        # new dist_arr contains only trajectories in clui_df
        clui_trajs = clusters[clui][2]
        clui_dist_arr = dist_arr[clui_trajs, :][:, clui_trajs]
        
        for sec_n_clu in range(n_start, n_end+1):
            # column names
            n_clu_list.append(meti + '_' + str(sec_n_clu))
            
            cluster = AgglomerativeClustering(n_clusters=sec_n_clu, 
                                              affinity='precomputed', linkage='complete')
            clu_results = cluster.fit(clui_dist_arr)
            labels[:, count] = clu_results.labels_
            count += 1
    
    # output pd.DataFrame
    cluster_2nd_df = pd.DataFrame(data=labels, columns=n_clu_list)
    # add tag, name, migr_status, TrajID to cluster_2nd_df
    cluster_2nd_df = clui_df.join(cluster_2nd_df)
    
    return cluster_2nd_df

def cmptSihouette(cluster_2nd_nc_df, clui):
    start_col = 6  # start with the 6th column  
    col_names = cluster_2nd_nc_df.columns.tolist()
    
    sih_score_list = []
    for coli in range(start_col, cluster_2nd_nc_df.shape[1]):     
        # (2nd) metric and n_clusters
        col_name = col_names[coli]
        index_ = col_name.index('_')
        metric_2nd = col_name[:index_]
        n_clusters_2nd = int(col_name[(index_+1):])
        
        # distance array
        # read precomputed distance array
        file_name = "mid_migr_lat_30N_tv_" + metric_2nd + "_arr"
        file_object = open(file_name, 'rb')
        dist_arr = pickle.load(file_object)
        # new dist_arr contains only trajectories in clui_df
        clui_trajs = cluster_2nd_nc_df.loc[:, 'TrajID'].to_list()
        clui_dist_arr = dist_arr[clui_trajs, :][:, clui_trajs]
    
        # compute sihouette score
        labels = cluster_2nd_nc_df.iloc[:, coli]
        sih_score = metrics.silhouette_score(clui_dist_arr, labels, metric='precomputed')
        sih_score_list.append(sih_score)
    
    # arrange the silhouette score list
    sih_score_list_arranged = []
    dist_metrics = ['frechet', 'dtw', 'hausdorff', 'lcss', 'nwed']
    # number of clusters tried
    n_metrics = 5
    n_clu_trials = int((cluster_2nd_nc_df.shape[1] - start_col) / n_metrics)
    for meti in range(len(dist_metrics)):
        new_list = [dist_metrics[meti], sih_score_list[meti*n_clu_trials:(meti+1)*n_clu_trials]]
        sih_score_list_arranged.append(new_list)
    
    return sih_score_list_arranged

In [ ]:
def bestSihZoomInCluster(metric, init_n_clu, clui, current_cluster_df):
    # zoom-in clustering
    data_df = zoomInNCluster(metric=metric, n_clusters=init_n_clu, clui=clui, 
                             current_cluster_df=current_cluster_df)
    data_reorder_df = Reorder_cluster_df(data_df, list(range(6, data_df.shape[1])))
    
    # compute sihouette coefficient
    sih = cmptSihouette(data_reorder_df, clui=clui)
    
    # find the metric with the highest sihouette coefficient
    metric_highest_sih = []
    for meti in sih:
        metric_highest_sih.append(max(meti[1]))
    high_mt_idx = metric_highest_sih.index(max(metric_highest_sih))
    if high_mt_idx == 0:
        high_metric = 'frechet'
    elif high_mt_idx == 1:
        high_metric = 'dtw'
    elif high_mt_idx == 2:
        high_metric = 'hausdorff'
    elif high_mt_idx == 3:
        high_metric = 'lcss'
    elif high_mt_idx == 4:
        high_metric = 'nwed'
    # n_clusters starts with 2
    high_n_clu = sih[high_mt_idx][1].index(metric_highest_sih[high_mt_idx]) + 2
    
    # clustering results that have the highest sihouette coefficient
    col_names = data_reorder_df.columns.to_list()[:6]
    mt_highest_sih = high_metric + '_' + str(high_n_clu)
    col_names.append(mt_highest_sih)
    high_sih_df = data_reorder_df.loc[:, col_names]
    
    # highest sihouette
    highest_sih = max(metric_highest_sih)
    
    return high_sih_df, mt_highest_sih, highest_sih

In [ ]:
def recurCluster_SC(current_cluster_df):
    # parameters for zoom-in clustering
    prev_mt_highest_sih = current_cluster_df.columns.to_list()[6]
    index_ = prev_mt_highest_sih.index('_')
    metric = prev_mt_highest_sih[:index_]
    init_n_clu = int(prev_mt_highest_sih[(index_+1):])
    clui = current_cluster_df.loc[0, prev_mt_highest_sih]
    
    # zoom-in clustering
    high_sih_df, mt_highest_sih, highest_sih = bestSihZoomInCluster(metric, init_n_clu, 
                                                       clui, current_cluster_df)
    index_ = mt_highest_sih.index('_')
    high_n_clu = int(mt_highest_sih[(index_+1):])
    
    # update output_list
    output_list_update = []
    for ci in range(high_n_clu):
        output_list_update.append(high_sih_df.loc[high_sih_df[mt_highest_sih] == ci]
                                  .reset_index(drop=True))
    
    return output_list_update, highest_sih, mt_highest_sih

def hrchyCluster_SC(dist_met, init_n_clu, clui, current_cluster_df=cluster_df):
    # sihouette records for each step of clustering
    sih_records = []
    # metric records
    met_records = []
    
    # zoom-in clustering
    high_sih_df, mt_highest_sih, highest_sih = bestSihZoomInCluster(dist_met, 
                                                                    init_n_clu, 
                                                                    clui, current_cluster_df)
    index_ = mt_highest_sih.index('_')
    high_n_clu = int(mt_highest_sih[(index_+1):])
    sih_records.append(highest_sih)
    met_records.append(mt_highest_sih)
    
    # output
    output_records = []
    output_list = []
    for ci in range(high_n_clu):
        output_list.append(high_sih_df.loc[high_sih_df[mt_highest_sih] == ci]
                           .reset_index(drop=True))
    hrchyi_list = []  # clusters of trajectories at that hierarchy
    for li in output_list:
        hrchyi_list.append(li.loc[:, 'TrajID'].to_list())
    output_records.append(hrchyi_list)
    
    # check the highest SC
    # to decide whether to continue zoom-in clustering
    global_highest_sih = highest_sih
    while global_highest_sih > sc_th:
        global_highest_sih = -1  # the highest silh at that hierarchy
        
        new_hrchyi_sih_records = []
        new_hrchyi_met_records = []
        
        # output list will change later
        orig_output_list = copy.deepcopy(output_list)
        # check whether the cluster needs further clustering
        updated_output_list = []
        for clui_id, clui_trajs in enumerate(hrchyi_list):
            if len(clui_trajs) <= 2:
                updated_output_list.append(output_list[clui_id])
                continue
            
            current_cluster_df = orig_output_list[clui_id]
            # zoom-in clustering
            output_list_update, highest_sih, mt_highest_sih = recurCluster_SC(current_cluster_df)
            
            # check whether this further clustering makes sense
            if highest_sih >= sc_th:  # clustering makes sense
                # update output_list
                updated_output_list.extend(output_list_update)
                
                # record silh and met
                new_hrchyi_sih_records.append(highest_sih)
                new_hrchyi_met_records.append(mt_highest_sih)
            
            else: 
                updated_output_list.append(output_list[clui_id])
        
            # update the global_highest_sih
            if highest_sih > global_highest_sih:
                global_highest_sih = highest_sih
        
        output_list = copy.deepcopy(updated_output_list)
        
        # chech whether the new hierarchy needs to be recorded
        if global_highest_sih >= sc_th:  
            # record trajs at that hierarchy
            new_hrchyi_list = []
            for li in output_list:
                new_hrchyi_list.append(li.loc[:, 'TrajID'].to_list())
            output_records.append(new_hrchyi_list)
        
            # record silh and met at that hierarchy
            sih_records.append(new_hrchyi_sih_records)
            met_records.append(new_hrchyi_met_records)
        
        # update the hierarchy list to begin the next iteration
        hrchyi_list = copy.deepcopy(new_hrchyi_list)
    
    return output_records, sih_records, met_records

In [ ]:
sc_th = 0.5

frechet_2_0_hrchy, frechet_2_0_hrchy_sih, frechet_2_0_hrchy_met = hrchyCluster_SC(
    dist_met='frechet', init_n_clu=2, clui=0, current_cluster_df=cluster_df)
frechet_2_1_hrchy, frechet_2_1_hrchy_sih, frechet_2_1_hrchy_met = hrchyCluster_SC(
    dist_met='frechet', init_n_clu=2, clui=1, current_cluster_df=cluster_df)

dtw_3_0_hrchy, dtw_3_0_hrchy_sih, dtw_3_0_hrchy_met = hrchyCluster_SC(
    dist_met='dtw', init_n_clu=3, clui=0, current_cluster_df=cluster_df)
dtw_3_1_hrchy, dtw_3_1_hrchy_sih, dtw_3_1_hrchy_met = hrchyCluster_SC(
    dist_met='dtw', init_n_clu=3, clui=1, current_cluster_df=cluster_df)

hausdorff_2_0_hrchy, hausdorff_2_0_hrchy_sih, hausdorff_2_0_hrchy_met = hrchyCluster_SC(
    dist_met='hausdorff', init_n_clu=2, clui=0, current_cluster_df=cluster_df)

lcss_3_0_hrchy, lcss_3_0_hrchy_sih, lcss_3_0_hrchy_met = hrchyCluster_SC(
    dist_met='lcss', init_n_clu=3, clui=0, current_cluster_df=cluster_df)
lcss_3_1_hrchy, lcss_3_1_hrchy_sih, lcss_3_1_hrchy_met = hrchyCluster_SC(
    dist_met='lcss', init_n_clu=3, clui=1, current_cluster_df=cluster_df)

nwed_2_0_hrchy, nwed_2_0_hrchy_sih, nwed_2_0_hrchy_met = hrchyCluster_SC(
    dist_met='nwed', init_n_clu=2, clui=0, current_cluster_df=cluster_df)

### Characterize clustering results

Try to interpret the clustering results with environmental variables obtained from Movebank.  Eamaine the clusters that are mostly agreed on, i.e., the results obtained starting with Frechet or DTW ro_up.

In [ ]:
# Kolmogorov-Smirnov (KS) test
env_vars = ['Movebank Thermal Uplift (from ECMWF)',
            'ECMWF Interim Full Daily SFC Temperature (2 m above Ground)',
            'MODIS Land Vegetation Indices 250m 16d Aqua NDVI',
            'Movebank Orographic Uplift (from ASTER DEM and ECMWF)',      
            'ECMWF ERA5 SL Wind (10 m above Ground U Component)',
            'ECMWF ERA5 SL Wind (10 m above Ground V Component)',
            'ECMWF ERA5 SL Total Precipitation']

In [ ]:
def Clu_Env_KS_test(cluster_lables, env_var_idx, migr_state, pts_df=pts_df, sim_pts_df=sim_pts_df):
    '''
    Compare the distribution of env vars in sim pts and org pts for each cluster.

    Parameters
    ----------
    cluster_lables : list
        The last element of it is the hierarchical clustering result.
    env_var_idx : list
        A list of environmental variable indexes.
    migr_state : str
        Fall or spring migration
    pts_df : pd.df, optional
        Original pts. The default is pts_df.
    sim_pts_df : pd.df, optional
        Simulation (background) pts. The default is sim_pts_df.

    Returns
    -------
    ks_results : list
        A list of test results together with the environmental variable names.
        e.g., [env_var1, KstestResult(statistic=0.444356027159..., pvalue=0.038850140086...)]
    '''
    # environmental variables
    env_vars = [pts_df.columns.to_list()[i] for i in env_var_idx]
    
    clu_trajs = []  # all clustered trajectories
    for clui in cluster_lables:
        clu_trajs.extend(clui)
    clu_pts = []  # all pts in clustered trajectories
    for tji in clu_trajs:
        clu_pts.extend(trajPtID[tji][1])
    
    ks_results = []  # output
    
    for clu_count, labelj in enumerate(cluster_lables):
        # KS test output
        d_stats = np.zeros((len(env_vars), 2))
        p_values = np.zeros((len(env_vars), 2))
        
        # trajectory points in the cluster
        labelj_pts = []
        for traji in labelj:
            labelj_pts.extend(trajPtID[traji][1])
        
        # cluster years
        clu_years = []
        for tji in clu_trajs:
            tji_pt0 = trajPtID[tji][1][0]
            pti_dt_obj = datetime.strptime(pts_df.loc[tji_pt0, 'study_local_timestamp'],
                               '%m/%d/%Y %H:%M:%S')
            pts_year = pti_dt_obj.year
            if pts_year not in clu_years:
                clu_years.append(pts_year)
        clu_years.sort()
        
        # simulated points (background) year
        clu_year_idx = [years.index(i) for i in clu_years]
        sim_pts_in_clu_years = []
        for yeari in clu_year_idx:
            sim_pts_in_clu_years.extend(year_sim_pts[yeari][1])
        clu_sim_pts_df = sim_pts_df.loc[sim_pts_in_clu_years]
        
        for env_count, envi in enumerate(env_vars):
            # values of the environmental variable of the points in the cluster
            labelj_envi = pts_df.loc[labelj_pts, envi].to_numpy()
            labelj_envi = labelj_envi[~np.isnan(labelj_envi)]  # remove nan
            
            if len(labelj_envi) == 0:  # missing values
                continue
            
            # simulated points (background) season
            clu_season_bool = clu_sim_pts_df['Migr_state'] == migr_state
            clu_season_sim_pts_df = clu_sim_pts_df.loc[clu_season_bool]
            sim_envi = clu_season_sim_pts_df.loc[:, envi].to_numpy()
            sim_envi = sim_envi[~np.isnan(sim_envi)]  # remove nan
        
            # KS test with the background
            labelj_envi_ks = stats.kstest(labelj_envi, sim_envi, N=len(labelj))
            d_stats[env_count, 0] = labelj_envi_ks[0]
            p_values[env_count, 0] = labelj_envi_ks[1]
           
        # results
        clu_ks_df = pd.DataFrame({
            'cluster': [(clu_count+1) for i in range(len(env_vars))],
            'env_var': env_vars,
            'stat_back': d_stats[:,0],
            'p-value_back': p_values[:, 0]
                               })
        ks_results.append([labelj, clu_ks_df])
    
    return ks_results

In [ ]:
fall_ks_results = Clu_Env_KS_test(cluster_lables=dtw_3_0_hrchy[-1], 
                                  env_var_idx=[19, 22, 21, 20, 25, 26, 27], 
                                  migr_state='Fall')
spring_ks_results = Clu_Env_KS_test(cluster_lables=dtw_3_1_hrchy[-1], 
                                    env_var_idx=[19, 22, 21, 20, 25, 26, 27], 
                                    migr_state='Spring')

In [ ]:
def Pair_Env_KS_test(cluster_lables, env_var_idx, migr_state, pts_df=pts_df, sim_pts_df=sim_pts_df):
    '''
    Pairwise KS test
    '''
    # environmental variables
    env_vars = [pts_df.columns.to_list()[i] for i in env_var_idx]
    
    clu_trajs = []  # all clustered trajectories
    for clui in cluster_lables:
        clu_trajs.extend(clui)
    clu_pts = []  # all pts in clustered trajectories
    for tji in clu_trajs:
        clu_pts.extend(trajPtID[tji][1])
    
    ks_results = []  # output
    
    for envi in env_vars:
        # KS test output
        p_values = np.ones((len(cluster_lables), len(cluster_lables)))
        
        for clu_count, labelj in enumerate(cluster_lables):
            # trajectory points in the cluster
            labelj_pts = []
            for traji in labelj:
                labelj_pts.extend(trajPtID[traji][1])
            
            # values of the environmental variable of the points in the cluster
            labelj_envi = pts_df.loc[labelj_pts, envi].to_numpy()
            labelj_envi = labelj_envi[~np.isnan(labelj_envi)]  # remove nan
            
            if len(labelj_envi) == 0:  # missing values
                continue
            
            for clu_count2, labelk in enumerate(cluster_lables):
                if clu_count == clu_count2:
                    continue
                
                # trajectory points in the cluster
                labelk_pts = []
                for traji in labelk:
                    labelk_pts.extend(trajPtID[traji][1])
                
                # values of the environmental variable of the points in the cluster
                labelk_envi = pts_df.loc[labelk_pts, envi].to_numpy()
                labelk_envi = labelk_envi[~np.isnan(labelk_envi)]  # remove nan
                
                if len(labelk_envi) == 0:  # missing values
                    continue
                
                # KS test with other clusters
                labelj_k_ks = stats.kstest(labelj_envi, labelk_envi, N=len(labelj))
                p_values[clu_count, clu_count2] = labelj_k_ks[1]
        
        ks_results.append([envi, p_values])
    
    return ks_results

In [ ]:
fall_pair_ks = Pair_Env_KS_test(cluster_lables=dtw_3_0_hrchy[-1], 
                                  env_var_idx=[19, 22, 21, 20, 25, 26, 27], 
                                  migr_state='Fall')
spring_pair_ks = Pair_Env_KS_test(cluster_lables=dtw_3_1_hrchy[-1], 
                                  env_var_idx=[19, 22, 21, 20, 25, 26, 27], 
                                  migr_state='Spring')

#### Jensen-Shannon distance

In [ ]:
def btw_clu_jsd(cluster_lables, env_var_idx):
    '''
    Between-cluster JSD
    '''
    env_vars = [pts_df.columns.to_list()[i] for i in env_var_idx]
    jsd_output = []
    
    for envi in env_vars:
        jsd_mat = np.zeros((len(cluster_lables), len(cluster_lables)))
        
        for clu_count, labelj in enumerate(cluster_lables):
            # trajectory points in the cluster
            labelj_pts = []
            for traji in labelj:
                labelj_pts.extend(trajPtID[traji][1])

            # values of the environmental variable of the points in the cluster
            labelj_envi = pts_df.loc[labelj_pts, envi].to_numpy()
            # remove nan
            labelj_envi = labelj_envi[~np.isnan(labelj_envi)]
            
            if len(labelj_envi) == 0:
                continue
            
            # range
            hist_range = (min(labelj_envi), max(labelj_envi))
            
            # probability vector
            labelj_envi_prob_vec = np.histogram(labelj_envi, bins=100, range=hist_range)[0] / len(labelj_envi)
            
            for clu_count2, labelk in enumerate(cluster_lables):
                # check whether they are the same cluster
                if clu_count2 == clu_count:
                    continue
                
                # trajectory points in cluster k
                labelk_pts = []
                for traji in labelk:
                    labelk_pts.extend(trajPtID[traji][1])
                labelk_envi = pts_df.loc[labelk_pts, envi].to_numpy()
                
                # remove nan
                labelk_envi = labelk_envi[~np.isnan(labelk_envi)]
                
                if len(labelk_envi) == 0:
                    continue
                
                # probability vector
                labelk_envi_prob_vec = np.histogram(labelk_envi, bins=100, range=hist_range)[0] / len(labelk_envi)
                # update jsd_mat
                jsd_jk = distance.jensenshannon(labelj_envi_prob_vec, labelk_envi_prob_vec)
                jsd_mat[clu_count, clu_count2] = jsd_jk
                jsd_mat[clu_count2, clu_count] = jsd_jk
            
        envi_list = [envi, jsd_mat]
        jsd_output.append(envi_list)
    
    return jsd_output

In [ ]:
env_var_idx = [19, 22, 21, 20, 25, 26, 27]
spring_jsd = btw_clu_jsd(cluster_lables=spring_clusters, env_var_idx=env_var_idx)

temp_mean_jsd = np.mean(spring_jsd[3][1][np.nonzero(spring_jsd[3][1])])
ndvi_mean_jsd = np.mean(spring_jsd[2][1][np.nonzero(spring_jsd[2][1])])

## Visualization

In [ ]:
sns.set_theme()
sns.set_context("paper")

In [ ]:
def plotHrchyCluter_SC(hrchy_results, fig_title):
    plt.figure(figsize=(16,6))
    x = [str(item) for sublist in hrchy_results[-1] for item in sublist]
    plt.plot(x, [0 for item in x], color=(0,0,0,0))  # transparent

    clus = []  # store the clusters
    for count, hrchyi in enumerate(hrchy_results):
        ci_y = len(hrchy_results) - count - 1
        for ci in hrchyi:
            if len(ci) == 1:  # only 1 trajectory
                plt.vlines(str(ci[0]), ci_y, ci_y+1, color='blue')
                clus.append(ci)
            else:
                # find the index in the string list
                x_idx = np.zeros((len(ci), 2)).astype(int)
                x_idx[:, 0] = ci
                for rowi in range(len(x_idx)):
                    x_idx[rowi, 1] = x.index(str(x_idx[rowi, 0]))
                x_idx_sort = x_idx[np.argsort(x_idx[:, 1])]

                plt.vlines(str(x_idx_sort[0, 0]), ci_y, ci_y+1, color='blue')
                plt.vlines(str(x_idx_sort[-1, 0]), ci_y, ci_y+1, color='blue')

                if ci not in clus:
                    ci_x = [str(item) for item in ci]
                    ci_y_list = [ci_y+1 for item in ci]
                    plt.plot(ci_x, ci_y_list, color='blue')
                    clus.append(ci)
    
    # the overarching cluster
    plt.vlines(x[0], len(hrchy_results), len(hrchy_results)+1, color='blue')
    plt.vlines(x[-1], len(hrchy_results), len(hrchy_results)+1, color='blue')
    plt.plot(x, [len(hrchy_results)+1 for item in x], color='blue')
    

    yticks = range(len(hrchy_results)+2)
    plt.yticks(yticks, yticks[::-1])
    plt.xlabel('Trajectory ID')
    plt.ylabel('Hierarchy')
    plt.title(fig_title)

In [ ]:
plotHrchyCluter_SC(frechet_2_0_hrchy, 'frechet_2_0')
plotHrchyCluter_SC(frechet_2_1_hrchy, 'frechet_2_1')

plotHrchyCluter_SC(dtw_3_0_hrchy, 'dtw_3_0')
plotHrchyCluter_SC(dtw_3_1_hrchy, 'dtw_3_1')

plotHrchyCluter_SC(hausdorff_2_0_hrchy, 'hausdorff_2_0')

plotHrchyCluter_SC(lcss_3_0_hrchy, 'lcss_3_0')
plotHrchyCluter_SC(lcss_3_1_hrchy, 'lcss_3_1')

plotHrchyCluter_SC(nwed_2_0_hrchy, 'nwed_2_0')

In [ ]:
def Plot_env_distr_bw(cluster_lables, env_var_idx, migr_state, bw=0.3, pts_df=pts_df, sim_pts_df=sim_pts_df):
    '''
    Plot the distribution of the environmental variables of vairous cluters 
        with simulated points as the baseline.
    '''
    env_vars = [pts_df.columns.to_list()[i] for i in env_var_idx]
    n_env_vars = len(env_vars)
    nrows = np.ceil(n_env_vars / 2)
    count = 1
    
    for envi in env_vars:
        plt.subplot(nrows, 2, count)
        
        # range of x
        xmin = min(pts_df[envi])
        xmax = max(pts_df[envi])
        dx = 0.2 * (xmax - xmin) # add a 20% margin, as the kde is wider than the data
        xmin -= dx
        xmax += dx
        x = np.linspace(xmin, xmax, 500)
        
        # simulated points
        if migr_state == 'Fall':
            fall_sim_pts_df = sim_pts_df.loc[sim_pts_df['Migr_state'] == 'Fall']
            sim_envi = fall_sim_pts_df.loc[:, envi].to_numpy()
        elif migr_state == 'Spring':
            spring_sim_pts_df = sim_pts_df.loc[sim_pts_df['Migr_state'] == 'Spring']
            sim_envi = spring_sim_pts_df.loc[:, envi].to_numpy()
        
        # remove nan
        sim_envi = sim_envi[~np.isnan(sim_envi)]
        # kernel
        sim_kernel = stats.gaussian_kde(sim_envi, bw_method=bw)
        # KDE
        kde_back_x = sim_kernel(x)

        for clu_count, labelj in enumerate(cluster_lables):
            # trajectory points in the cluster
            labelj_pts = []
            for traji in labelj:
                labelj_pts.extend(trajPtID[traji][1])

            # values of the environmental variable of the points in the cluster
            labelj_envi = pts_df.loc[labelj_pts, envi].to_numpy()
            labelj_envi = labelj_envi[~np.isnan(labelj_envi)]  # remove nan
            
            if len(labelj_envi) == 0:
                continue
    
            # kernel j
            kernelj = stats.gaussian_kde(labelj_envi, bw_method=bw)
            kdej_x = kernelj(x)

            # density plot
            color_palette = sns.color_palette("Paired")
            if clu_count < 12:
                color =  color_palette[clu_count] # RGB alpha
            else:
                count12 = clu_count - 12
                color_palette = sns.color_palette("Set2")
                color =  color_palette[count12]
            plt.plot(x, kdej_x, color=color, label='Cluster_'+str(clu_count+1))

        plt.xlabel(envi)
        
        if count % 2 == 1:
            plt.ylabel("Probability density")

        count += 1
    
    handles, labels = plt.gca().get_legend_handles_labels()
    labels, ids = np.unique(labels, return_index=True)
    sort_idx = np.argsort(ids)
    ids = ids[sort_idx]
    labels = labels[sort_idx]
    handles = [handles[i] for i in ids]
    plt.figlegend(labels, loc='center right')

In [ ]:
env_var_idx = [19, 22, 21, 20, 25, 26, 27]

# fall migration
plt.figure(figsize=(16,12))
Plot_env_distr_bw(dtw_3_0_hrchy[-1], env_var_idx, migr_state='Fall', bw=0.3)
# plt.suptitle("Fall migration", fontsize=16)
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(wspace=0.11)
plt.subplot(4,2,5)  # precipitation
plt.xlim(-0.001, 0.002)
plt.suptitle('Fall migration')

# spring migration
plt.figure(figsize=(16,12))
Plot_env_distr_bw(dtw_3_1_hrchy[-1], env_var_idx, migr_state='Spring', bw=0.3)
# plt.suptitle("Spring migration", fontsize=16)
plt.subplots_adjust(top=0.95)
plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(wspace=0.11)
plt.subplot(4,2,5)
plt.xlim(-0.001, 0.002)
plt.suptitle('Spring migration')